**1.1** Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

``` postgresql
SELECT COUNT(*)  
FROM stackoverflow.posts  
WHERE post_type_id = 1 AND (score > 300 OR favorites_count >= 100)
```

**1.2** Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

``` postgresql
SELECT ROUND(AVG(cnt))  
FROM (  
SELECT creation_date::date,  
       COUNT(id) AS cnt  
FROM stackoverflow.posts  
WHERE post_type_id = 1 AND creation_date::date BETWEEN '2008-11-01' AND '2008-11-18'  
GROUP BY creation_date::date  
) AS per  
```

**1.3** Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

```postgresql
SELECT COUNT(DISTINCT(u.id))  
FROM stackoverflow.users AS u  
JOIN stackoverflow.badges AS b ON u.id = b.user_id  
WHERE u.creation_date::date = b.creation_date::date  
```

**1.4** Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

```postgresql
SELECT COUNT(DISTINCT(post_id))  
FROM stackoverflow.votes AS v  
WHERE post_id IN (  
    SELECT p.id  
    FROM stackoverflow.posts AS p  
    JOIN stackoverflow.users AS u ON p.user_id = u.id  
    WHERE u.display_name = 'Joel Coehoorn'   
   )   
```

**1.5** Выгрузите все поля таблицы `vote_types`. Добавьте к таблице поле `rank`, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю `id`.

```postgresql
SELECT *,
       RANK() OVER(ORDER BY id DESC)
FROM stackoverflow.vote_types
ORDER BY id
```

**1.6** Отберите 10 пользователей, которые поставили больше всего голосов типа `Close`. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

```postgresql
SELECT DISTINCT(user_id) AS us,
       COUNT(v.id) OVER(PARTITION BY v.user_id) AS cnt
FROM stackoverflow.votes AS v
JOIN stackoverflow.vote_types AS vt ON v.vote_type_id = vt.id
WHERE vt.id = 6
ORDER BY cnt DESC, us DESC
LIMIT 10
```

**1.7** Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.

Отобразите несколько полей:
- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.

Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

```postgresql
WITH us_b AS (
    SELECT DISTINCT(u.id) AS us,
    COUNT(b.id) OVER(PARTITION BY u.id) AS cnt
    FROM stackoverflow.users AS u
    JOIN stackoverflow.badges AS b ON u.id = b.user_id
    WHERE b.creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
    ORDER BY cnt DESC, us ASC
    LIMIT 10
    )
    
SELECT *,
       DENSE_RANK() OVER(ORDER BY cnt DESC)
FROM us_b
```

**1.8** Сколько в среднем очков получает пост каждого пользователя?

Сформируйте таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.


```postgresql
SELECT title, 
       user_id,
       score,
       ROUND(AVG(score) OVER(PARTITION BY user_id))
FROM stackoverflow.posts
WHERE score != 0 AND title IS NOT NULL
```

**1.9** Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

```postgresql
SELECT title
FROM stackoverflow.posts AS p
WHERE title IS NOT NULL AND user_id IN (
    SELECT DISTINCT(u.id) AS us
    FROM stackoverflow.users AS u
    JOIN stackoverflow.badges AS b ON u.id = b.user_id
    GROUP BY u.id
    HAVING COUNT(b.id) > 1000
    )
```

**1.10** Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с количеством просмотров меньше либо равным нулю не должны войти в итоговую таблицу.


```postgresql
SELECT DISTINCT(id),
       views,
       CASE
           WHEN views >= 350 THEN 1
           WHEN views < 350 AND views >= 100 THEN 2
           WHEN views < 100 THEN 3
       END
FROM stackoverflow.users
WHERE location LIKE '%Canada%' AND views != 0
```

**1.11** Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

```postgresql
WITH gr AS (
    SELECT DISTINCT(id) AS us,
           views AS v,
           CASE
               WHEN views >= 350 THEN 1
               WHEN views < 350 AND views >= 100 THEN 2
               WHEN views < 100 THEN 3
           END AS grp
    FROM stackoverflow.users
    WHERE location LIKE '%Canada%' AND views != 0
    ),
    tab_1  AS (
        SELECT *,
               DENSE_RANK() OVER(PARTITION BY grp ORDER BY v DESC) AS rn       
        FROM gr
        )
SELECT us, grp, v
FROM tab_1
WHERE rn = 1
ORDER BY v DESC, us
```

**1.12** Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

```postgresql
SELECT ROW_NUMBER() OVER(),
       COUNT(id),
       SUM(COUNT(id)) OVER(ORDER BY creation_date::date)
       
FROM stackoverflow.users
WHERE creation_date::date BETWEEN '2008-11-01' AND '2008-11-30'
GROUP BY creation_date::date
```

**1.13** Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

```postgresql
SELECT DISTINCT(user_id),
       MIN(p.creation_date) OVER(PARTITION BY user_id)-u.creation_date
FROM stackoverflow.posts AS p 
JOIN stackoverflow.users AS u ON p.user_id = u.id
```

**2.1** Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

```postgresql
SELECT DATE_TRUNC('month', creation_date)::date,
       SUM(views_count) AS s
FROM stackoverflow.posts
WHERE EXTRACT(YEAR FROM creation_date::date) = 2008
GROUP BY DATE_TRUNC('month', creation_date)::date
ORDER BY s DESC
```

**2.2** Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений `user_id`. Отсортируйте результат по полю с именами в лексикографическом порядке.

```postgresql
SELECT display_name,
       COUNT(DISTINCT(user_id))
FROM stackoverflow.users AS u
JOIN stackoverflow.posts AS p ON u.id = p.user_id
WHERE p.post_type_id = 2 
      AND p.creation_date::date >= u.creation_date::date 
      AND p.creation_date::date <= (u.creation_date::date + INTERVAL '1 month')
GROUP BY u.display_name
HAVING COUNT(p.id) > 100
ORDER BY display_name
```

**2.3** Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

```postgresql
SELECT DATE_TRUNC('month', creation_date)::date AS dt,
       COUNT(id) AS cnt
FROM stackoverflow.posts
WHERE EXTRACT(YEAR FROM creation_date::date) = 2008
      AND user_id IN (
      SELECT u.id
    FROM stackoverflow.users AS u
    JOIN stackoverflow.posts AS p ON u.id = p.user_id
    WHERE u.creation_date::date BETWEEN '2008-09-01' AND '2008-09-30'
      AND DATE_TRUNC('month', p.creation_date::date) = '2008-12-01')
GROUP BY DATE_TRUNC('month', creation_date)::date
ORDER BY dt DESC
```

**2.4** Используя данные о постах, выведите несколько полей:
- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумма просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

```postgresql
SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER(PARTITION BY user_id ORDER BY creation_date)
       
FROM stackoverflow.posts
ORDER BY user_id, creation_date
```

**2.5** Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

```postgresql
WITH u AS (
SELECT DISTINCT(user_id) AS us,
       creation_date::date AS dt,
       COUNT(creation_date) OVER(PARTITION BY user_id, creation_date::date) AS cnt
FROM stackoverflow.posts
WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07'
ORDER BY user_id
    ),
    us_cn AS (
SELECT DISTINCT(us) AS us,
       COUNT(dt) OVER(PARTITION BY us) AS cnt
FROM u
        )
        SELECT ROUND(AVG(cnt))
        FROM us_cn
```

**2.6** На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
- Номер месяца.
- Количество постов за месяц.
- Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.

```postgresql
WITH mon AS (
    SELECT EXTRACT(MONTH FROM creation_date::date) AS dt,
           COUNT(id) AS cnt
    FROM stackoverflow.posts
    WHERE creation_date::date BETWEEN '2008-09-01' AND '2008-12-31'
    GROUP BY EXTRACT(MONTH FROM creation_date::date)
    ORDER BY dt
    ),
    nx AS (
        SELECT *,
               LAG(cnt) OVER(ORDER BY dt) AS nxt
        FROM mon
        )
SELECT dt,
       cnt,
       ROUND((cnt::numeric/nxt::numeric - 1) * 100, 2)
FROM nx
```

**2.7** Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведите данные его активности за октябрь 2008 года в таком виде:
- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

```postgresql
WITH usr AS (
    SELECT DISTINCT(user_id) AS us,
           COUNT(id) OVER(PARTITION BY user_id) AS cnt
    FROM stackoverflow.posts
    ORDER BY cnt DESC
    LIMIT 1
    )
SELECT DISTINCT(EXTRACT(WEEK FROM creation_date::date)),
       MAX(creation_date) OVER(PARTITION BY EXTRACT(WEEK FROM creation_date::date))
FROM stackoverflow.posts AS p
JOIN usr ON p.user_id = usr.us
WHERE creation_date::date BETWEEN '2008-10-01' AND '2008-10-31'
```